# 2. Georeferencing

This tutorial takes on the challenge of georeferencing the imagery. To georeference, you need:

a. Synchronized navigation data covering the time of the transect (or sub-transect chunk) in the form of 3 positions and 3 orientations.

b. A camera model (we use *.XML files) describing boresight angles, lever arms, focal length, principal point, and distortion coefficients.

c. A digital elevation/surface model, either in as orthographic "*.tif" file, or as a triangular mesh, "*.ply" file.

In [13]:
"""We start by some imports"""
# Standard python library
import configparser
import sys
import os

# Third party libraries
import numpy as np
import pyvista as pv

module_path = 'C:/Users/haavasl/VSCodeProjects/hyperspectral_toolchain/src/'
if module_path not in sys.path:
    sys.path.append(module_path)

# Local resources
from lib import parsing_utils
from scripts import visualize
from scripts.modulate_config import prepend_data_dir_to_relative_paths
from lib.parsing_utils import reformat_h5_embedded_data_h5
from scripts.gis_tools import dem_2_mesh
from scripts.geometry import CameraGeometry, CalibHSI
from lib.parsing_utils import Hyperspectral


In [14]:

# Definitions of folder paths
DISK_NAME = 'D:/'
DATA_DIR = DISK_NAME + 'HyperspectralDataAll/HI/2022-08-31-060000-Remoy-Specim/'

INPUT_DIR = DATA_DIR + 'Input/'
H5_DIR = INPUT_DIR + 'H5/'

OUTPUT_DIR = DATA_DIR + 'Output/'
config_file = DATA_DIR + 'configuration.ini'

# Set the data directory for the mission (locally where the data is stored)
prepend_data_dir_to_relative_paths(config_path=config_file, DATA_DIR = DATA_DIR)



## a. Reading the poses

In [15]:


# Use updated config file to reformat the position and orientation data.
config = configparser.ConfigParser()
config.read(config_file)
"""What happens under the hood is that 3xN array of ECEF positions and euler angle orientations are converted to a specific geocentric (ECEF) format: 
They are also interpolated to the times of the hyperspectral frames."""
config = reformat_h5_embedded_data_h5(config=config,
                                              config_file=config_file)




True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3
True
Hello3


In [16]:
# After processing, the 3D positions for each transect chunk can be found under the hierarchical path
pos_h5_path = config['HDF.processed_nav']['position_ecef']
quat_h5_path = config['HDF.processed_nav']['quaternion_ecef']

print('Path to ECEF quaternions is {0} \nPath to ECEF positions is {1}'.format(pos_h5_path,quat_h5_path))

# NB, note that to avoid rounding errors the positions in h5 file are subtracted a globally known offset:
# If is is desired to inspect the data, h5 files 


Path to ECEF quaternions is processed/nav/position_ref_ecef 
Path to ECEF positions is processed/nav/quaternion_ref_ecef


# b. Reading in the digital elevation model
To use the georeferencing framework for ray casting on arbitrary 3D surfaces, we employ 3D triangular meshes. Therefore, in the case that you only have digital elevation model as tif, you will need to convert the model.

In [17]:

# The input projected DEM file. IMPORTANT: heights must be given with respect to ellipsoid
file_path_dem = config['Absolute Paths']['dempath']

# The output geocentric 3D model, which uses same CRS as poses
file_path_3d_model = config['Absolute Paths']['modelpath']






Note that some DEM's are given with respect to mean sea level (i.e. geoid). In that case adding the geoid height is necessary as:
´´´
dem_total = dem_terrain_wrt_geoid + dem_geoid
´´´

* dem_geoid: Geoid dems can be found at https://www.agisoft.com/downloads/geoids/ both global models (e.g. EGM 2008) and local models (e.g. NN2000 for Norway)
* dem_terrain_wrt_geoid: Terrain models are often supplied by regional authority, norwegian mapping authority at https://hoydedata.no/LaserInnsyn2/. Alternatively, terrain models are supplied by aerial drones from SeaBee infrastructure (either as DSM or DEM or DTM)
* For ocean color it is rational to use the geoid and tide level.
* At present, refractive georeferencing is not implemented as it is inefficient for the adjustment procedure. When it is added as functionality, the DEMs must be corrected for refraction, and water depth utilized.
* At present, Uncorrected photo-DEMs can be used as-is, while true bathymetry data e.g. LiDAR should be transformed 
$$
h_{e,b}^{*} = h_{e,wl} + (h_{e,b} - h_{e,wl})\frac{1}{n_{w}}
$$
* Where $h_{e,b}^{*}$ is the ellipsoid height in the transformed bathymetry DEM to be used onward, $h_{e,b}$ is the ellipsoid heights in the true bathymetry, $h_{e,wl}$ is the ellipsoid height of the water line (available from NMA) and $n_{w}$ is the refractive index of water, often 1.34 is used.


In [18]:
# No need for this step if a mesh model file already exists
dem_2_mesh(path_dem=file_path_dem, model_path=file_path_3d_model, config=config)

DEM projected EPSG Code: 25832
Conversion completed.
Mesh geocentric EPSG Code: 4936


For a more intuitive understanding, visualization of navigation data and positions/orientations is useful. It is also good for a sanity check to see that the data is interpreted correctly. For a regular airborne vehicle, the blue arrows should point towards the surface

In [19]:

%matplotlib qt

# For aerial imaging, looking at data in a east, north, up (ENU) frame makes most sense.
# Pressing the "Top(-Z)" option is most illustrative as it gives an orthographic view with north upward, and east to the right.
# The red arrow: forward/x-axis on Specim vehicle, aligned with vehicle's direction of motion and opposite of y-axis of HSI.
# The green arrow: starboard/right/y-axis on Specim vehicle. Aligns with the x-axis of HSI.
# The blue arrow: downward/z-axis on Specim vehicle. Aligns with the x-axis of HSI. Should point perfectly perpendicular to terrain surface.
# Other available options for frame are NED and ECEF.


#visualize.show_mesh_camera(config, show_mesh = True, show_pose = True, ref_frame = 'ENU')


# c. Reading in camera model
The camera model's first component is a rigid body transform from 3 lever arm parameters and 3 boresight rotation angles:

$R_{h}^{e}(t) = R_{i}^{e}(t) R_{h}^{i}\\
p_{h/e}^{e} = p_{i/e}^{e} + R_{i}^{e}(t)p_{h/i}^{i}$

In [20]:
CAM_MODEL_PATH_XML = config['Absolute Paths']['hsicalibfile']
import xmltodict
import numpy as np
from scipy.spatial.transform import Rotation

with open(CAM_MODEL_PATH_XML, 'r', encoding='utf-8') as xml_file:
    my_xml = xml_file.read()
    xml_dict = xmltodict.parse(my_xml)

# For ease, print the XML file
print(xml_dict)

# Rotations
rx = float(xml_dict['calibration']['rx'])
ry = float(xml_dict['calibration']['ry'])
rz = float(xml_dict['calibration']['rz'])
eul_ZYX = np.array([rz, ry, rx])
R_h_i = Rotation.from_euler('ZYX', eul_ZYX, degrees=False)


# Translations describing p_{i/h} i.e. vector from HSI to IMU.
tx = float(xml_dict['calibration']['tx'])
ty = float(xml_dict['calibration']['ty'])
tz = float(xml_dict['calibration']['tz'])
p_i_h = np.array([tx, ty, tz]).reshape((-1))
p_h_i = -p_i_h

# Example of composition. Assume a ECEF position [0, 0, 0] and rotation matrix at identity I_3
p_i_e = np.array([0, 0, 0]).reshape((-1))
R_i_e = Rotation.from_matrix(np.eye(3))

# The equations above thus becomes
R_h_e = R_i_e*R_h_i
p_h_e = p_i_e + R_i_e.apply(p_h_i)

# Set the precision for printing
np.set_printoptions(precision=3, suppress=True)
print(R_h_e.as_matrix())
print(p_h_e)

hsi_x_basis = R_h_i.as_matrix()[:,0] # Defaults to [0 -1 0]
hsi_y_basis = R_h_i.as_matrix()[:,1]
hsi_z_basis = R_h_i.as_matrix()[:,2]

# TODO: Make process simpler to avoid confusion between typical computer vision frames (as used for HSI) and the typical navigation frames (i.e. vehicle body)

# Note that writing, or modifying parameters can be done as:
#xml_dict['calibration']['tx'] = value
#with open(file_name_cal_xml, 'w') as fd:
#    fd.write(xmltodict.unparse(xml_dict))

{'calibration': {'rx': '0.0', 'ry': '0.0', 'rz': '-1.5707963267948966', 'tx': '0', 'ty': '0', 'tz': '0', 'f': '754.9669306613682', 'cx': '255.0099175768686', 'k1': '-72.31616804110381', 'k2': '-389.5781973543412', 'k3': '4.075384334827561', 'width': '512'}}
[[ 0.  1.  0.]
 [-1.  0. -0.]
 [-0.  0.  1.]]
[0. 0. 0.]


# (Optional) Tide compensation

To compensate for tide effects, it is necessary to either use an on-site measurement or data from national authorities. Below we illustrate with data from the norwegian mapping authorities (downloaded from online with vertival reference NN2000)

In [32]:
path_tide = os.path.join('..', '02_Georeferencing', 'data', 'tidevann_nn2000_NMA.txt')

# d. Start the actual georeferencing.

In [21]:
# Paths to 3D mesh ply file 
path_mesh = config['Absolute Paths']['modelPath']

# Directory of H5 files
dir_r = config['Absolute Paths']['h5Dir']

# The path to the XML file
hsi_cal_xml = config['Absolute Paths']['HSICalibFile']

# Maximal allowed ray length
max_ray_length = float(config['General']['maxRayLength'])

mesh = pv.read(path_mesh)

h5_files = sorted(os.listdir(dir_r))

# For illustration, let us use 1st file:
h5_file = h5_files[0]

print(h5_file)

2022-08-31-060000-Remoy-Specim_transectnr_0_chunknr_0.h5


In [22]:
from scripts import georeference_mod as grf
# Read h5 file

hyp = Hyperspectral(dir_r + h5_file, config)

# We can here access the interpolated position, orientation data, timestamps and the position offset (used to avoid rounding errors)
pos_ref_ecef = hyp.pos_ref
quat_ref_ecef = hyp.quat_ref
time_pose = hyp.pose_time
pos0 = hyp.pos0

# Meaning that pos_ref_ecef_tot = pos_ref_ecef + pos0

# Using the cal file, we can define lever arm, boresight and local ray geometry (in dictionary)
intrinsic_geometry_dict = grf.cal_file_to_rays(filename_cal=hsi_cal_xml, config=config)

# Using both intrinsic and extrinsic geometry, we can define all ray directions
hsi_geometry = grf.define_hsi_ray_geometry(pos_ref_ecef, quat_ref_ecef, time_pose, pos0, intrinsic_geometry_dict)

# hsi_geometry is an object holding the necessary geometry for performing the intersection:
hsi_geometry.intersectWithMesh(mesh = mesh, max_ray_length=max_ray_length)



# Writing an RGB point cloud version of the datacube (only for visualization, not very useful)
hsi_geometry.writeRGBPointCloud(config = config, hyp = hyp, transect_string = h5_file.split('.')[0])

# Computes the view angles in the local NED. Computationally intensive as local NED is defined for each intersection
hsi_geometry.compute_view_directions_local_tangent_plane()

# Computes the sun angles in the local NED. Computationally intensive as local NED is defined for each intersection
hsi_geometry.compute_sun_angles_local_tangent_plane()

# () Compute the tide level for each measurement, based on a tide file downloaded from the norwegian mapping authority
hsi_geometry.compute_tide_level(path_tide, tide_format = 'NMA')

#visualize.show_projected_hsi_points(HSICameraGeometry=hsi_geometry, config=config, transect_string = h5_file.split('.')[0])

# TODO We should also compute the water depth from e.g. LIDAR, although this could perhaps be computed easier for rectified data
# Simplest be a function 


# Writing intersection geometry to the h5 file
grf.write_intersection_geometry_2_h5_file(hsi_geometry=hsi_geometry, config = config, h5_filename=dir_r + h5_file)

visualize.show_projected_hsi_points(HSICameraGeometry=hsi_geometry, config=config, transect_string = h5_file.split('.')[0])







Starting Ray Tracing
Ray traced 1024000 rays on a mesh with 100674 cells in 0.6638753414154053 seconds
Finished ray tracing


In [23]:
"""
# Example calculation of solar angle, verified against NOAA: https://gml.noaa.gov/grad/solcalc/
from datetime import datetime

lon = hsi_geometry.lons.mean()
print(lon)

lat = hsi_geometry.lats.mean()
print(lat)

time_unix = hsi_geometry.time.mean()
print(time_unix)

print(datetime.utcfromtimestamp(time_unix))

phi_s, theta_s = CameraGeometry.calculate_sun_directions(lon, lat, 0, time_unix)


print(f'{phi_s} is the azimuth and {90-theta_s} is the elevation angle')"""


"\n# Example calculation of solar angle, verified against NOAA: https://gml.noaa.gov/grad/solcalc/\nfrom datetime import datetime\n\nlon = hsi_geometry.lons.mean()\nprint(lon)\n\nlat = hsi_geometry.lats.mean()\nprint(lat)\n\ntime_unix = hsi_geometry.time.mean()\nprint(time_unix)\n\nprint(datetime.utcfromtimestamp(time_unix))\n\nphi_s, theta_s = CameraGeometry.calculate_sun_directions(lon, lat, 0, time_unix)\n\n\nprint(f'{phi_s} is the azimuth and {90-theta_s} is the elevation angle')"

In [24]:
# Do this for all files
for h5_file in h5_files:
    hyp = Hyperspectral(dir_r + h5_file, config)

    # We can here access the interpolated position, orientation data, timestamps and the position offset (used to avoid rounding errors)
    pos_ref_ecef = hyp.pos_ref
    quat_ref_ecef = hyp.quat_ref
    time_pose = hyp.pose_time
    pos0 = hyp.pos0

    # Meaning that pos_ref_ecef_tot = pos_ref_ecef + pos0

    # Using the cal file, we can define lever arm, boresight and local ray geometry (in dictionary)
    intrinsic_geometry_dict = grf.cal_file_to_rays(filename_cal=hsi_cal_xml, config=config)

    # Using both intrinsic and extrinsic geometry, we can define all ray directions
    hsi_geometry = grf.define_hsi_ray_geometry(pos_ref_ecef, quat_ref_ecef, time_pose, pos0, intrinsic_geometry_dict)

    # hsi_geometry is an object holding the necessary geometry for performing the intersection:
    hsi_geometry.intersectWithMesh(mesh = mesh, max_ray_length=max_ray_length)

    # Calculate the earth-sun vector per scanline and the view directions in NED
    hsi_geometry.compute_view_directions_local_tangent_plane()
    # Writing intersection geometry to the h5 file
    grf.write_intersection_geometry_2_h5_file(hsi_geometry=hsi_geometry, config = config, h5_filename=dir_r + h5_file)

'# Do this for all files\nfor h5_file in h5_files:\n    hyp = Hyperspectral(dir_r + h5_file, config)\n\n    # We can here access the interpolated position, orientation data, timestamps and the position offset (used to avoid rounding errors)\n    pos_ref_ecef = hyp.pos_ref\n    quat_ref_ecef = hyp.quat_ref\n    time_pose = hyp.pose_time\n    pos0 = hyp.pos0\n\n    # Meaning that pos_ref_ecef_tot = pos_ref_ecef + pos0\n\n    # Using the cal file, we can define lever arm, boresight and local ray geometry (in dictionary)\n    intrinsic_geometry_dict = grf.cal_file_to_rays(filename_cal=hsi_cal_xml, config=config)\n\n    # Using both intrinsic and extrinsic geometry, we can define all ray directions\n    hsi_geometry = grf.define_hsi_ray_geometry(pos_ref_ecef, quat_ref_ecef, time_pose, pos0, intrinsic_geometry_dict)\n\n    # hsi_geometry is an object holding the necessary geometry for performing the intersection:\n    hsi_geometry.intersectWithMesh(mesh = mesh, max_ray_length=max_ray_length)

To use the point-cloud georeferences further, they 